## 고고학 연구동향 연구(가제)
* [DBpia-Crawler](https://github.com/ChanToRe/DBpia-Crawler)를 통해 수집된 고고학 논문을 분석하여 2000년 이후 고고학 연구 동향을 살핌
* 워드클라우드 시각화 기법과 토픽 모델링 기법을 활용하여 연구동향을 분석

### 설치
```bash
pip install tqdm
pip install pandas
pip install matplotlib
pip install konlpy
pip install wordcloud
```

### 진행
1. 크롤러 개발 (2022/01/26 완료)
2. 데이터 체크 및 한자 번역 (2022/01/31 완료)
3. 명사 추출 및 빈도 체크 (2022/02/01 완료) 
4. 워드 클라우드 적용 확인 (2022/02/02 완료)
5. 불용어 사전, 고유명사 사전 제작 (2022/02/04 진행중~) [#1](https://github.com/ChanToRe/Archaeological-research-trend-analysis/issues/1#issue-1123805974) 참조
6. Okt -> Mecab 전환 (2022/02/05 완료)
7. 사회 연결망 분석 구현(2022/02/07 완료)

---

In [ ]:
import pandas as pd
import networkx as nx
import hanja
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from konlpy.tag import Mecab
from wordcloud import WordCloud, STOPWORDS
import numpy as np
from collections import Counter
import csv
import re

In [ ]:
#데이터 로드 & 한글 깨짐 방지
plt.rcParams['font.family'] = 'AppleGothic'
data = pd.read_csv('./Data-before.csv')
df = pd.DataFrame(data)

In [ ]:
#제목 한자 번역
before_title = df[['title']] #제목 추출
list_before_title = before_title.values.tolist() #제목 리스트화
after_title = [] #번역 후 제목 리스트


for i in tqdm(range(len(list_before_title))):
    trans = hanja.translate(list_before_title[i], "substitution")
    after_title.append(trans)

In [ ]:
trans_result = df.assign(translate_title = after_title)
DB = pd.DataFrame(trans_result)
sort_DB = DB.sort_values(by=['date'])
#DB.to_excel("~/Desktop/result.xlsx")

In [ ]:
#고고학 논문 발간 수 그래프
sort_DB["date"].value_counts().sort_index().plot(kind='bar')
plt.rcParams["figure.figsize"] = (20,10)
plt.title("고고학 논문 발간 수", fontsize=20)
plt.xlabel("연도", fontsize=20)
plt.ylabel("수량", fontsize=20)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.grid()
plt.show()

---

In [ ]:
#Mecab 생성
mecab = Mecab()
wc = WordCloud(width=1000, height=1000, random_state=0, max_words=50, max_font_size=300 ,background_color="white", font_path=r'~/Library/Fonts/AppleGothic.ttf')

In [ ]:
#불용어 처리
def clean_text(row):
    text = row['translate_title']
    txt = re.sub('[a-zA-z]','',text)
    txxt = re.sub('[^가-힣a-z]', ' ', txt)
    text = mecab.morphs(txxt)
    stopwords_path = "/Users/jch/Desktop/stopword.txt"
    with open(stopwords_path, encoding='UTF-8') as file:
        stopwords = file.readlines()
    stopwords = [x.strip() for x in stopwords]
    text = [t for t in text if t not in stopwords]
    return text

trans_result['translate_title'] = trans_result.apply(clean_text, axis=1)

In [ ]:
#분기별 구분
date_trash = trans_result[trans_result['date'] < 2000] #2000년대 이전

#2000년대 발간 논문
date_zero = trans_result[(trans_result['date'] >= 2000) & (trans_result['date'] < 2010)]
date_zero_nx = trans_result[(trans_result['date'] >= 2000) & (trans_result['date'] < 2010)]
date_zero_title = pd.DataFrame(date_zero['translate_title'])
date_zero_title.to_csv("./date_zero_title.csv", sep = "\t", index=False)
zero = open("./date_zero_title.csv", 'r', encoding="UTF-8")
data_zero = zero.read()
zero.close()

#2010년대 발간 논문
date_ten = trans_result[(trans_result['date'] >= 2010) & (trans_result['date'] < 2020)]
date_ten_nx = trans_result[(trans_result['date'] >= 2010) & (trans_result['date'] < 2020)] 
date_ten_title = pd.DataFrame(date_ten['translate_title'])
date_ten_title.to_csv("./date_ten_title.csv", sep = "\t", index=False)
ten = open("./date_ten_title.csv", 'r', encoding="UTF-8")
data_ten = ten.read()
ten.close()

#2020년대 발간 논문
date_twenty = trans_result[trans_result['date'] > 2019]
date_twenty_nx = trans_result[trans_result['date'] > 2019] 
date_twenty_title = pd.DataFrame(date_twenty['translate_title'])
date_twenty_title.to_csv("./date_twenty_title.csv", sep = "\t", index=False)
twenty = open("./date_twenty_title.csv", 'r', encoding="UTF-8")
data_twenty = twenty.read()
twenty.close()

In [ ]:
#발간 논문 명사 빈도 카운트(워드클라우드)
def noun_count(data):
    noun = mecab.nouns(data)
    count = Counter(noun)
    noun_list = count.most_common(50)
    plt.axis("off")
    plt.imshow(wc.generate_from_frequencies(dict(noun_list)))

#네트워크 분석 수행 모듈
def network_analysis(date_nx):
    G = nx.Graph()
    edge_list = []

    for trans_dict in date_nx['translate_title']:
        trans = list(trans_dict)
        num_trans = len(trans)
        if num_trans > 0 :
            for j in range(num_trans-1):
                for k in range(j+1, num_trans):
                    edge_list += [tuple(sorted([trans[j], trans[k]]))]
    edges = list(Counter(edge_list).items())

    G = nx.Graph((x, y, {'weight': v}) for (x, y), v in edges)

    font_path='/Users/jch/Library/Fonts/AppleGothic.ttf'
    font_name = font_manager.FontProperties(fname=font_path).get_name()
    rc('font', family=font_name)
    return G

#네트워크 분석 그래프 모듈
def network_graph(G):

    nx.Graph()

    pr = nx.pagerank(G)
    nsize = np.array([v for v in pr.values()])
    nsize = 2000 * (nsize - min(nsize)) / (max(nsize) - min(nsize))

    plt.figure(figsize=(80,80))
    pos = nx.kamada_kawai_layout(G)

    nx.draw_networkx_nodes(G, pos,
                    node_shape = "o",
                    node_color=range(len(G)),
                    cmap=plt.cm.Blues_r,
                    node_size=25 * nsize)

    nx.draw_networkx_edges(G, pos,
                    style='solid',
                    width=5,
                    alpha=0.1,
                    edge_color='#808080')

    nx.draw_networkx_labels(G, pos,
                    font_size=20,
                    font_family=font_name)
    plt.show()

#연결중심성
def bet_centratlity(data, end):
    bet_cen = nx.betweenness_centrality(data)
    result = sorted(bet_cen.items(), key=lambda x:x[1], reverse=True)[0:end]
    return result

---

In [ ]:
G_zero = network_analysis(date_zero_nx)
network_graph(G_zero)

noun_count(data_zero)
    
bet_centratlity(G_zero, 10)